Import all modules you'll need. if you get an error like : no module "module_name" found; just run "pip install module_name" in empty cell.

In [1]:
%matplotlib Qt
%load_ext autoreload 
# to automaticaly reload all the modules that have been modified
%autoreload 2 
import os
from glob import glob
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
from scipy import interpolate
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, ShuffleSplit
from sklearn import svm, kernel_ridge, metrics, preprocessing
from PIL import Image, ExifTags, ImageFile
import visualize as vis
import preprocessing as pp
import processing as proc
import calculate as cc
import pandas as pd
import correction as corr
from read_WDF import read_WDF, get_exif
import warnings
warnings.filterwarnings('ignore')

For the moment, I haven't found a module in python that reads FTIR .map files, so you need to transform your file into .xlsx

### Reading

In [2]:
os.chdir('C:/Users/raoul.missodey/Desktop/Code_python')
files = glob('*.xlsx')
for i, j in enumerate(files):
    print(i, ') ', j)

0 )  FTIR-Carto-T140.xlsx
1 )  FTIR_Carto.xlsx
2 )  ~$FTIR-Carto-T140.xlsx
3 )  ~$FTIR_Carto.xlsx


In [4]:
filename = files[1] # choose the number of .xlsx file you want to use.
# load the file
da =  pd.read_excel(filename)

In [5]:
# show your data
da

,2499.332520,2503.189515,2507.046510,2510.903505,2514.760500,2518.617495,2522.474490,2526.331485,2530.188480,2534.045475,...,5473.075741,5476.932736,5480.789732,5484.646727,5488.503722,5492.360717,5496.217712,5500.074707,map_x,map_y
0,0.243704,0.233024,0.229345,0.243829,0.249446,0.253125,0.266522,0.274338,0.276199,0.268497,...,0.123744,0.116647,0.111874,0.115657,0.113502,0.104931,0.104030,0.104789,-38,-516
1,0.262143,0.253922,0.256896,0.260294,0.254070,0.261234,0.265178,0.268294,0.280731,0.290285,...,0.121235,0.115275,0.110587,0.111453,0.114556,0.108613,0.105503,0.108252,-36,-516
2,0.270031,0.271826,0.282452,0.276281,0.285322,0.300282,0.307268,0.295894,0.288382,0.307975,...,0.120943,0.114753,0.107939,0.111124,0.109730,0.105028,0.110412,0.115009,-34,-516
3,0.274932,0.268010,0.279422,0.283939,0.287064,0.294149,0.301576,0.301561,0.292915,0.296271,...,0.123692,0.121545,0.119733,0.122042,0.119663,0.115742,0.123059,0.128410,-32,-516
4,0.261806,0.260074,0.280358,0.296670,0.304425,0.294321,0.300976,0.303632,0.298022,0.310915,...,0.132134,0.132266,0.124586,0.125018,0.130221,0.126165,0.120978,0.122873,-30,-516
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331,0.361509,0.351403,0.349895,0.354733,0.358172,0.341385,0.322885,0.302312,0.298153,0.298315,...,0.440938,0.448375,0.452538,0.449839,0.434168,0.426187,0.431805,0.435515,-6,-486
332,0.375064,0.361907,0.346038,0.341450,0.334026,0.329911,0.325099,0.317773,0.310814,0.305808,...,0.455078,0.452365,0.440305,0.439770,0.433648,0.429405,0.432621,0.428284,-4,-486
333,0.366758,0.370629,0.362103,0.360901,0.345829,0.324359,0.324675,0.318105,0.327720,0.326116,...,0.452617,0.453760,0.444677,0.434099,0.434208,0.440098,0.446444,0.443024,-2,-486
334,0.321572,0.310914,0.339352,0.354631,0.330111,0.315210,0.310199,0.294964,0.276517,0.289435,...,0.433352,0.437045,0.445241,0.452322,0.454956,0.439398,0.429112,0.428607,0,-486


The first row of our data contains the Raman shifts and the last two columns are map_x and map_y. 

We are going to arrange it.

## Preprocessing

In [7]:
da_data = da.iloc[:,0:-2].values.copy()
ftir_mapx = da.iloc[:,-2].values.copy()
ftir_mapy = da.iloc[:,-1].values.copy()
raman_shifts = da.columns.values[:-2]
raman_shifts =  np.round([float(i) for i in da.columns.values[:-2]],1)

In [10]:
# run this cell to see all your spectra
vis.ShowSpectra(da_data, sigma = raman_shifts)

### Baseline Substraction

Two options to substract the baseline 

#### 1st option

In [ ]:
#  substract baseline : you have to modify lambda_value(smoothing parameter), p_value(the penalizing weighting factor)
# and lambda1_value(the smoothing parameter for the first derivative of the residual).
check_bline = vis.FindBaseline(da_data,sigma=raman_shifts)

In [ ]:
# run to see chosen values
print(f"p: {check_bline.p_val:.2e}, lam: {check_bline.lam_val:.2e},lam1: {check_bline.lam1_val:.2e} ")

In [ ]:
# calculate the basline for all spectrum
baseline = cc.baseline_ials(da_data, p=check_bline.p_val, lam=check_bline.lam_val, lam1= check_bline.lam1_val)

In [ ]:
# substraction
db = da_data.copy()
db.values = db.values - baseline

#### 2nd option

In [11]:
check_bline = vis.FindBaseline1(da_data,sigma=raman_shifts)

In [12]:
print(f"lam: {check_bline.lam_val:.2e}")

lam: 3.23e+07


In [14]:
# calculate the basline for all spectrum
baseline = cc.baseline_arpls(da_data, lam=check_bline.lam_val)

In [15]:
# substraction
db = da_data.copy()
db = db - baseline

### Normalization

In [16]:
# choose normalize method you want to apply : min_max, l1, l2, max, area, robust_scale 
db -= np.min(db, axis=-1, keepdims=True)
dn = pp.normalize(db, method="min_max")

## Processing

### PCA
Before doing the PCA, you need to know your scan_shape. After that, you can replace it in the "pp.svd_decomp" . You can also changed the "n_components"

In [17]:
# compute pca using svd decomposition
dsvd ,var,score ,l= pp.svd_decomp(da_data,components_sigma=raman_shifts, n_components=10, visualize_clean=1, visualize_components=1,visualize_err=1,scanshape=(16,21))

### NMF

The same thing for NMF decompostion

In [19]:
# Run this cell to see pure spectrum and pure contribution
dnm = proc.deconvolve_nmf(da_data,components_sigma=raman_shifts, n_components=4, visualize_components=1, visualize_compare=1, visualize_err=1,scanshape=(16,21))

Error increased above fractionalctol_increase (C iter). Exiting
